# Machine Learning - Examen Final

## Integrantes

*   Alumno 1: <font color='green'> 201622489 - Daniel Núñez Robinson </font><br>

## Importando librerías y datasets

In [173]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import *
from tensorflow.keras.utils import normalize
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randrange

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

from scipy.stats import mode

In [144]:
url1="https://raw.githubusercontent.com/JumpNShootMan/Modelos_ML/main/Riesgo_AccidenteCardiovascular.csv"
url2="https://raw.githubusercontent.com/JumpNShootMan/Modelos_ML/main/Prediccion_Ventas.csv"
url3="https://raw.githubusercontent.com/JumpNShootMan/Modelos_ML/main/Abandono.csv"

df1= pd.read_csv(url1, encoding="ISO-8859-1")
df2= pd.read_csv(url2, encoding = "ISO-8859-1")
df3= pd.read_csv(url3, encoding = "ISO-8859-1")

## Caso de Estudio – Diagnosticando Retinopatía Diabética

1. Descargue los archivos **csv** anexados.
2. Aplique los conocimientos de ingeniería de características, clasificación supervisada y/o no supervisada, análisis de datos, evaluación de resultados.
3. Documente sus métodos y elecciones (al menos 3 estimadores por ejercicio). Codifique su solución. Obtenga resultados y realice comparaciones. Indique por qué eligió los estimadores seleccionados. 


## Resolución

### Ejercicio 1: (6 puntos)

Identificar a las personas propensas a sufrir un accidente cardiovascular, según el archivo `Riesgo_AccidenteCardiovascular.csv`. En resultados y discusión, asegurese de incluir discusiones sobre la especificidad y sensibilidad (*sensitivity*) de sus estimadores.

#### Implementación

In [145]:
df1.head()

,id,genero,edad,hipertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,accidente cardiovascular
0,30669,Masculino,3.0,0,0,No,niños,Rural,95.12,18.0,NaN,0
1,30468,Masculino,58.0,1,0,Si,privado,Urbano,87.96,39.2,nunca,0
2,16523,Femenino,8.0,0,0,No,privado,Urbano,110.89,17.6,NaN,0
3,56543,Femenino,70.0,0,0,Si,privado,Rural,69.04,35.9,regularmente,0
4,46136,Masculino,14.0,0,0,No,nunca_trabajo,Rural,161.28,19.1,NaN,0


In [146]:
df1.dtypes

id                            int64
genero                       object
edad                        float64
hipertension                  int64
heart_disease                 int64
ever_married                 object
work_type                    object
Residence_type               object
avg_glucose_level           float64
bmi                         float64
smoking_status               object
accidente cardiovascular      int64
dtype: object

### Verificar que no haya errores a nivel de nombre para poder hacer one hot

In [147]:
df1.groupby('genero').size()
#df1.groupby('ever_married').size()
# df1.groupby('work_type').size()
# df1.groupby('Residence_type').size()
# df1.groupby('smoking_status').size()

genero
Femenino     25676
Masculino    17724
dtype: int64

In [148]:
for col in df1:
    print(df1[col].isnull().sum())

0
0
0
0
0
0
0
0
0
1462
13292
0


In [149]:
df1.shape

(43400, 12)

In [150]:
def oneHotEncoding(feature):
    onehot_encoded = pd.get_dummies(df1[feature], prefix=feature)
    del df1[feature]
    return df1.join(onehot_encoded)

In [151]:
df1 = oneHotEncoding('genero')
df1 = oneHotEncoding("smoking_status")
df1 = oneHotEncoding("ever_married")
df1 = oneHotEncoding("work_type")
df1 = oneHotEncoding("Residence_type")

In [153]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#df1.smoking_status = imputer.fit_transform(df1['smoking_status'].values.reshape(-1,1))
df1.bmi = imputer.fit_transform(df1['bmi'].values.reshape(-1,1))

In [154]:
df1 = df1.drop(columns=["id"])
df1.head()

,edad,hipertension,heart_disease,avg_glucose_level,bmi,accidente cardiovascular,genero_Femenino,genero_Masculino,smoking_status_frecuente,smoking_status_nunca,smoking_status_regularmente,ever_married_No,ever_married_Si,work_type_gubernamental,work_type_independiente,work_type_niños,work_type_nunca_trabajo,work_type_privado,Residence_type_Rural,Residence_type_Urbano
0,3.0,0,0,95.12,18.0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
1,58.0,1,0,87.96,39.2,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1
2,8.0,0,0,110.89,17.6,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1
3,70.0,0,0,69.04,35.9,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0
4,14.0,0,0,161.28,19.1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0


In [155]:
def standardization(feature):
    scaler = MinMaxScaler()
    return scaler.fit_transform(df1[feature].values.reshape(-1,1))

In [156]:
df1['edad'] = standardization('edad')
df1['bmi'] = standardization('bmi')
df1['avg_glucose_level'] = standardization('avg_glucose_level')

In [157]:
df1.head()

,edad,hipertension,heart_disease,avg_glucose_level,bmi,accidente cardiovascular,genero_Femenino,genero_Masculino,smoking_status_frecuente,smoking_status_nunca,smoking_status_regularmente,ever_married_No,ever_married_Si,work_type_gubernamental,work_type_independiente,work_type_niños,work_type_nunca_trabajo,work_type_privado,Residence_type_Rural,Residence_type_Urbano
0,0.035645,0,0,0.169964,0.090286,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
1,0.707031,1,0,0.139631,0.332571,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1
2,0.096680,0,0,0.236772,0.085714,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1
3,0.853516,0,0,0.059479,0.294857,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0
4,0.169922,0,0,0.450244,0.102857,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0


### Creación de datasets particionados para el entrenamiento

In [168]:
train, test = train_test_split(df1, test_size=0.2, random_state=42, shuffle=True)

x_train = train.iloc[:,train.columns != 'accidente cardiovascular']
y_train = train.iloc[:,train.columns == 'accidente cardiovascular']
x_test = test.iloc[:,test.columns != 'accidente cardiovascular']
y_test = test.iloc[:,test.columns == 'accidente cardiovascular']

print(x_train.shape)
print(x_test.shape)

(34720, 19)
(8680, 19)


In [170]:
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[8330  189]
 [ 149   12]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      8519
           1       0.06      0.07      0.07       161

    accuracy                           0.96      8680
   macro avg       0.52      0.53      0.52      8680
weighted avg       0.97      0.96      0.96      8680



#### Resultados y Discusión

<font color='green'> Discuta los **varios resultados obtenidos** por la selección de **diversos parámetros seleccionados**. Indique por que eligió los estimadores/modelos seleccionados. </font>

### Ejercicio 2:  (6 puntos)

Predecir las ventas que una empresa tendrá, según los datos del archivo `Prediccion_Ventas.csv`.

#### Implementación

In [10]:
df2.head()

,id,peso,grasa,visibilidad,tipo,precio minorista maximo,id tienda unica,inaguracion,tamanho,ubicacion,tipo de tienda,ventas
0,FDA15,9.30,Bajo,0.016047,Diariamente,249.8092,OUT049,1999,Mediano,Sede 1,Supermercado Tipo1,3735.1380
1,DRC01,5.92,Regular,0.019278,Bebidas Suaves,48.2692,OUT018,2009,Mediano,Sede 3,Supermercado Tipo2,443.4228
2,FDN15,17.50,Bajo,0.016760,Carne,141.6180,OUT049,1999,Mediano,Sede 1,Supermercado Tipo1,2097.2700
3,FDX07,19.20,Regular,0.000000,Frutas y Vegetales,182.0950,OUT010,1998,NaN,Sede 3,Abarrotes,732.3800
4,NCD19,8.93,Bajo,0.000000,Productos de casa,53.8614,OUT013,1987,Grande,Sede 3,Supermercado Tipo1,994.7052


#### Resultados y Discusión

<font color='green'> Discuta los **varios resultados obtenidos** por la selección de **diversos parámetros seleccionados**. Indique por que eligió los estimadores/modelos seleccionados. </font>

### Ejercicio 3: (8 puntos)

Utilizando el archivo `Abandono.csv`, identificar a los clientes con alta probabilidad de ser abandonar la empresa. Adicionalmente descubrir algún número de categorías que represente bien a los grupos de clientes en transición (desde los clientes fieles, hasta los que seguramente se irán).

#### Implementación

In [12]:
df3.head()

,id cliente,genero,estado civil,tercera edad,dependientes,meses de fidelidad,servicio telefónico,Multiples servicios,tipo de servicios de internet,seguridad de linea,...,proteccion del dispositivo,soporte técnico,StreamingTV,Streaming Películas,Contrato,facturacion electronica,metodo de pago,MonthlyCharges,cargos totales,abandono
0,7590-VHVEG,Femenino,0,Si,No,1,No,Sin serviico de teléfono,DSL,No,...,No,No,No,No,Mes a mes,Si,Electrónico,29.85,29.85,No
1,5575-GNVDE,Masculino,0,No,No,34,Si,No,DSL,Si,...,Si,No,No,No,Un anho,No,Correo,56.95,1889.5,No
2,3668-QPYBK,Masculino,0,No,No,2,Si,No,DSL,Si,...,No,No,No,No,Mes a mes,Si,Correo,53.85,108.15,Si
3,7795-CFOCW,Masculino,0,No,No,45,No,Sin serviico de teléfono,DSL,Si,...,Si,Si,No,No,Un anho,No,Transferencia,42.30,1840.75,No
4,9237-HQITU,Femenino,0,No,No,2,Si,No,Fibra optica,No,...,No,No,No,No,Mes a mes,Si,Electrónico,70.70,151.65,Si


In [174]:
df3.groupby('Multiples servicios').size()

Multiples servicios
No                          3390
Si                          2971
Sin serviico de teléfono     682
dtype: int64

#### Resultados y Discusión

<font color='green'> Discuta los **varios resultados obtenidos** por la selección de **diversos parámetros seleccionados**. Indique por que eligió los estimadores/modelos seleccionados. </font>